In [1]:
import sys
sys.path.append("..")
from src.decision_engine import DecisionEngine
from src.query_parser import Query_parser
from src.embeddings import EmbeddingsManager
import json

print("Initializing components...")

Initializing components...


In [2]:
# Initialize all components
parser=Query_parser()
em=EmbeddingsManager()
em.create_collection("policy_documents")
decision_engine=DecisionEngine()

print(f"✅ Using LLM: {decision_engine.provider} ({decision_engine.model})")


'(MaxRetryError('HTTPSConnectionPool(host=\'huggingface.co\', port=443): Max retries exceeded with url: /sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F89C6D3380>: Failed to resolve \'huggingface.co\' ([Errno 11001] getaddrinfo failed)"))'), '(Request ID: ea18fed3-02d6-48ff-9892-d31d7b22f307)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


✅ Using LLM: groq (llama-3.1-8b-instant)


In [3]:
print("Testing LLM connection...\n")
is_working = decision_engine.test_connection()

if is_working:
    print("\n✅ LLM is ready!")
else:
    print("\n❌ LLM connection failed. Check your setup.")

Testing LLM connection...


✅ LLM is ready!


In [4]:
print("="*80)
print("TEST 1: Simple Approved Case")
print("="*80 + "\n")

# Mock data
query_info = {
    'age': 46,
    'gender': 'male',
    'procedure': 'knee surgery',
    'location': 'Pune',
    'policy_duration_months': 3,
    'is_emergency': False
}

retrieved_docs = [
    "Knee surgery, hip replacement, and spinal procedures are covered at 80% of actual hospital bills. Coverage begins after 3 months of active policy for accident-related cases.",
    "Treatment in network hospitals in Pune, Mumbai, Bangalore, and Delhi are covered at 100% of scheduled amounts.",
    "Policyholders aged 18-50 are eligible for all surgical procedures without additional screening."
]

retrieved_metadata = [
    {'section': 'SURGICAL COVERAGE AND BENEFITS'},
    {'section': 'GEOGRAPHIC COVERAGE'},
    {'section': 'AGE ELIGIBILITY'}
]

decision = decision_engine.make_decision(query_info, retrieved_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision.approved else '❌ REJECTED'}")
print(f"Amount: ₹{decision.amount:,}" if decision.amount else "Amount: Not specified")
print(f"Confidence: {decision.confidence}")
print(f"\nReasoning:")
print(f"  {decision.reasoning}")
print(f"\nRelevant Clauses:")
for clause in decision.relevant_clauses:
    print(f"  - {clause}")
if decision.risk_factors:
    print(f"\nRisk Factors:")
    for risk in decision.risk_factors:
        print(f"  ⚠️  {risk}")


TEST 1: Simple Approved Case

Decision: ❌ REJECTED
Amount: Not specified
Confidence: medium

Reasoning:
  The policyholder's age (46) is within the eligible range (18-50), but the policy duration is less than 3 months, which is a requirement for elective surgeries. Additionally, the procedure (knee surgery) is covered, but the treatment location (Pune) is a Tier 1 city, which would typically result in 100% coverage, but the policy does not specify any additional requirements for Tier 1 cities.

Relevant Clauses:
  - AGE ELIGIBILITY
  - SURGICAL COVERAGE AND BENEFITS
  - GEOGRAPHIC COVERAGE

Risk Factors:
  ⚠️  Insufficient policy duration
  ⚠️  Missing information on Tier 1 city coverage


In [5]:
print("\n" + "="*80)
print("TEST 2: Rejection - Insufficient Policy Duration")
print("="*80 + "\n")

query_info_reject = {
    'age': 45,
    'gender': 'male',
    'procedure': 'knee surgery',
    'location': 'Mumbai',
    'policy_duration_months': 1,  # Too short!
    'is_emergency': False
}

decision2 = decision_engine.make_decision(query_info_reject, retrieved_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision2.approved else '❌ REJECTED'}")
print(f"Confidence: {decision2.confidence}")
print(f"\nReasoning:")
print(f"  {decision2.reasoning}")
print(f"\nRisk Factors:")
for risk in decision2.risk_factors:
    print(f"  ⚠️  {risk}")


TEST 2: Rejection - Insufficient Policy Duration

Decision: ❌ REJECTED
Confidence: high

Reasoning:
  The claim is rejected due to insufficient policy duration. According to the policy, coverage begins after 3 months of active policy for accident-related cases, and the policy duration is only 1 month. [SURGICAL COVERAGE AND BENEFITS]

Risk Factors:
  ⚠️  Insufficient policy duration
  ⚠️  Policy duration is only 1 month


In [6]:
print("\n" + "="*80)
print("TEST 3: Emergency Case - Immediate Coverage")
print("="*80 + "\n")

query_info_emergency = {
    'age': 28,
    'gender': 'male',
    'procedure': 'cardiac',
    'location': 'Delhi',
    'policy_duration_months': 1,
    'is_emergency': True  # Emergency!
}

emergency_docs = [
    "Emergency procedures covered immediately from policy start date with zero waiting period.",
    "Angioplasty, bypass surgery, and valve replacement covered at 90% of actual costs.",
    "Treatment in network hospitals in Pune, Mumbai, Bangalore, and Delhi are covered at 100% of scheduled amounts."
]

decision3 = decision_engine.make_decision(query_info_emergency, emergency_docs, retrieved_metadata)

print(f"Decision: {'✅ APPROVED' if decision3.approved else '❌ REJECTED'}")
print(f"Amount: ₹{decision3.amount:,}" if decision3.amount else "Amount: Not specified")
print(f"Confidence: {decision3.confidence}")
print(f"\nReasoning:")
print(f"  {decision3.reasoning}")



TEST 3: Emergency Case - Immediate Coverage

Decision: ✅ APPROVED
Amount: Not specified
Confidence: low

Reasoning:
  The claim is approved as it meets all the requirements. The patient is within the age eligibility range (18-50), the policy duration is sufficient for emergency procedures, the cardiac procedure is covered, and the treatment location is in a Tier 1 city (Delhi).


In [7]:
decisions = [
    {"test": "approved_case", "decision": decision.dict()},
    {"test": "rejection_case", "decision": decision2.dict()},
    {"test": "emergency_case", "decision": decision3.dict()}
]

with open('../data/processed/sample_decisions.json', 'w') as f:
    json.dump(decisions, f, indent=2)

print("\n✅ Sample decisions saved to data/processed/sample_decisions.json")


✅ Sample decisions saved to data/processed/sample_decisions.json
